In [1]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import matplotlib.pyplot as plt
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)

In [2]:
h1b_df=pd.read_csv('H-1B_Disclosure_Data_FY17.csv')

### DATA ANALYSIS

In [3]:
h1b_df['value'] = 1

top_employers = h1b_df.groupby('EMPLOYER_NAME',as_index=False).count()
top_employers = top_employers.sort_values('value',ascending= False)[['EMPLOYER_NAME','value']][0:15]

top_employers_job = h1b_df.groupby(['EMPLOYER_NAME','CASE_STATUS'],as_index=False).count()

top_employers_job=top_employers_job[top_employers_job.EMPLOYER_NAME.isin(top_employers.EMPLOYER_NAME)]

tab1 = go.Bar(x=top_employers_job[top_employers_job.CASE_STATUS == 'CERTIFIED'].sort_values('value',ascending= False)['EMPLOYER_NAME'].values,y=top_employers_job[top_employers_job.CASE_STATUS == 'CERTIFIED'].sort_values('value',ascending= False)['value'].values,name='CERTIFIED',marker=dict(
        color='rgb(102,255,102)', line=dict(color='rgb(102,255,102)',width=1.5,)) )

tab2 = go.Bar(x=top_employers_job[top_employers_job.CASE_STATUS == 'DENIED'].sort_values('value',ascending= False)['EMPLOYER_NAME'].values,y=top_employers_job[top_employers_job.CASE_STATUS == 'DENIED'].sort_values('value',ascending= False)['value'].values,name='DENIED',marker=dict(
        color='rgb(204,0,0)', line=dict(color='rgb(204,0,0)',width=1.5,)) )
data = [tab1,tab2]

layout = go.Layout(barmode='stack',title="Top 15 Employers - Proportion of Certified and Denied Applications")
fig =go.Figure(data,layout)
iplot(fig)

In [4]:
states_category = h1b_df.groupby('EMPLOYER_STATE',as_index=False).count()[['EMPLOYER_STATE','value']].sort_values('value',ascending=False)

scl = [[0.0, 'rgb(255,153,204)'],[0.2, 'rgb(255,102,178)'],[0.4, 'rgb(255,51,153)'],[0.6, 'rgb(255,0,127)'],[0.8, 'rgb(204,0,102)'],[1.0, 'rgb(153,0,76)']]

data=[dict(type='choropleth',colorscale = scl, locations = states_category.EMPLOYER_STATE, z = states_category.value,
    locationmode = 'USA-states',marker = dict(line = dict (color = 'rgb(192,192,192)',width = 2) ),colorbar = dict(
    title = "Number of applications"))]

layout= dict(title="US States - H1B Applications",geo = dict(scope='usa',projection=dict( type='albers usa' ),showlakes = True,lakecolor = 'rgb(255, 255, 255)'),)
 
fig = dict( data=data, layout=layout )
iplot(fig)

In [5]:
top_jobs = h1b_df.groupby('JOB_TITLE',as_index=False).count()[['JOB_TITLE','value']].sort_values('value',ascending=False)[0:15]
tab1 = go.Bar(x=top_jobs.JOB_TITLE.values,y=top_jobs.value.values,name='jobtitle',marker=dict(color='rgb(158,202,225)', line=dict(color='rgb(8,48,107)',width=1.5,)) )
layout = go.Layout(dict(title= "Top 15 Jobs - Visa Applications",yaxis=dict(title="Num of applications")))
fig =go.Figure([tab1],layout)
iplot(fig)

In [6]:
top_jobs_visa = h1b_df.groupby(['JOB_TITLE','CASE_STATUS'],as_index=False).count()
top_jobs_visa = top_jobs_visa[top_jobs_visa.JOB_TITLE.isin(top_jobs.JOB_TITLE)]

tab1 = go.Bar(x=top_jobs_visa[top_jobs_visa.CASE_STATUS == 'CERTIFIED'].sort_values('value',ascending= False)['JOB_TITLE'].values,y=top_jobs_visa[top_jobs_visa.CASE_STATUS == 'CERTIFIED'].sort_values('value',ascending= False)['value'].values,name='CERTIFIED',marker=dict(
        color='green', line=dict(color='green',width=1.5,)) )

tab2 = go.Bar(x=top_jobs_visa[top_jobs_visa.CASE_STATUS == 'DENIED'].sort_values('value',ascending= False)['JOB_TITLE'].values,y=top_jobs_visa[top_jobs_visa.CASE_STATUS == 'DENIED'].sort_values('value',ascending= False)['value'].values,name='DENIED',marker=dict(
        color='red', line=dict(color='red',width=1.5,)) )
data = [tab1,tab2]

layout = go.Layout(barmode='stack',title="Top 15 Jobs - Proportion of Certified and Denied Applications")
fig =go.Figure(data,layout)
iplot(fig)

In [7]:
h1b_df['CASE_SUBMITTED'] =pd.to_datetime(h1b_df['CASE_SUBMITTED'])
h1b_df['YEAR'] = h1b_df.CASE_SUBMITTED.apply(lambda x: x.year)

In [8]:
tab1 = go.Scatter(
    x=h1b_df.groupby('YEAR').mean().index,y=h1b_df.groupby('YEAR').mean().PREVAILING_WAGE,marker = dict(size = 10, color = 'rgba(255, 0, 0, .9)',
    line = dict(width = 2,)))

layout = go.Layout(dict(title= "AVERAGE ANNUAL SALARY OVER YEARS",xaxis=dict(title="YEARS"),yaxis=dict(title="AVERAGE ANNUAL SALARY")))
fig = go.Figure(data=[tab1],layout=layout)
iplot(fig)

### DATA WRANGLING

In [9]:
display(h1b_df.VISA_CLASS.value_counts())
display(h1b_df.EMPLOYER_COUNTRY.value_counts())
display(h1b_df.PW_UNIT_OF_PAY.value_counts())
display(h1b_df.CASE_STATUS.value_counts())
display(h1b_df.YEAR.value_counts())

H-1B               610304
E-3 Australian      12157
H-1B1 Singapore      1254
H-1B1 Chile           935
Name: VISA_CLASS, dtype: int64

UNITED STATES OF AMERICA    528132
CANADA                           7
AUSTRALIA                        2
CHINA                            1
CAMBODIA                         1
Name: EMPLOYER_COUNTRY, dtype: int64

Year         585301
Hour          38779
Month           327
Week            140
Bi-Weekly        57
Name: PW_UNIT_OF_PAY, dtype: int64

CERTIFIED              545694
CERTIFIED-WITHDRAWN     49704
WITHDRAWN               20772
DENIED                   8480
Name: CASE_STATUS, dtype: int64

2017    505974
2016    103018
2015      9771
2014      5578
2013       295
2012        12
2011         2
Name: YEAR, dtype: int64

#### Considering the large number of data points and outliers, we decided to limit our focus on the H-1B applications, with yearly pay, submitted in United States in the year 2016, and landed either in a CERTIFIED or a DENIED status.

In [10]:
h1b_df = h1b_df[h1b_df.VISA_CLASS == 'H-1B']
h1b_df = h1b_df[h1b_df.EMPLOYER_COUNTRY == 'UNITED STATES OF AMERICA']
h1b_df = h1b_df[h1b_df.PW_UNIT_OF_PAY == 'Year']
h1b_df = h1b_df[(h1b_df.CASE_STATUS == 'CERTIFIED') | (h1b_df.CASE_STATUS == 'DENIED') ]
h1b_df = h1b_df[h1b_df.YEAR == 2016]

In [11]:
#Column added by calculating the days of employment

h1b_df['EMPLOYMENT_START_DATE'] =pd.to_datetime(h1b_df['EMPLOYMENT_START_DATE'])
h1b_df['EMPLOYMENT_END_DATE'] =pd.to_datetime(h1b_df['EMPLOYMENT_END_DATE'])
h1b_df['DAYS_OF_EMPLOYMENT'] = (h1b_df['EMPLOYMENT_END_DATE'] - h1b_df['EMPLOYMENT_START_DATE']).dt.days

#### Feature Selections

In [12]:
to_select = ['CASE_STATUS', 'DAYS_OF_EMPLOYMENT','EMPLOYER_NAME','EMPLOYER_STATE','AGENT_REPRESENTING_EMPLOYER','JOB_TITLE','FULL_TIME_POSITION',
            'PREVAILING_WAGE','H1B_DEPENDENT','WILLFUL_VIOLATOR','WORKSITE_STATE','NAICS_CODE']
h1b_df = h1b_df[to_select]

In [13]:
h1b_df.isnull().sum()[h1b_df.isnull().sum() > 0]

DAYS_OF_EMPLOYMENT             3
EMPLOYER_NAME                  3
AGENT_REPRESENTING_EMPLOYER    1
WORKSITE_STATE                 2
dtype: int64

#### Data Points with Null value are removed

In [14]:
h1b_df = h1b_df[h1b_df['DAYS_OF_EMPLOYMENT'].notnull()]
h1b_df = h1b_df[h1b_df['EMPLOYER_NAME'].notnull()]
h1b_df = h1b_df[h1b_df['AGENT_REPRESENTING_EMPLOYER'].notnull()]
h1b_df = h1b_df[h1b_df['WORKSITE_STATE'].notnull()]

#### Outliers for the feature PREVAILING_WAGE are removed

In [15]:
dum = h1b_df[(h1b_df.FULL_TIME_POSITION == 'Y')]
ind1 = dum[(dum.PREVAILING_WAGE > 200000) | (dum.PREVAILING_WAGE < 20000)].index
h1b_df = h1b_df.drop(ind1,axis=0)

In [16]:
state_map = {"AL":"South","AR":"South","AZ":"West","CA":"West","CO":"West","CT":"Northeast","DC":"South","DE":"South",
"FL":"South","GA":"South","HI":"West","IA":"Midwest","ID":"West","IL":"Midwest","IN":"Midwest","KS":"Midwest","KY":"South",
"LA":"South","MA":"Northeast","MD":"South","ME":"Northeast","MI":"Midwest","MN":"Midwest","MO":"Midwest","MS":"South","MT":"West",
"NC":"South","ND":"Midwest","NE":"Midwest","NH":"Northeast","NJ":"Northeast","NM":"West","NV":"West","NY":"Northeast","OH":"Midwest",
"OK":"South","OR":"West","PA":"Northeast","RI":"Northeast","SC":"South","SD":"Midwest","TN":"South","TX":"South","UT":"West","VA":"South",
"VT":"Northeast","WA":"West","WI":"Midwest","WV":"South","WY":"West" }

ind_map = { "11":"Agriculture, Forestry, Fishing and Hunting","15":"Agriculture, Forestry, Fishing and Hunting",
           "19":"Agriculture, Forestry, Fishing and Hunting","21":"Mining, Quarrying, and Oil and Gas Extraction",
           "22":"Utilities","23":"Construction","24":"Construction","27":"Construction","31":"Manufacturing",
           "32":"Manufacturing","33":"Manufacturing","42":"Wholesale Trade","44":"Retail Trade","45":"Retail Trade",
           "48":"Transportation and Warehousing","49":"Transportation and Warehousing","51":"Information",
           "52":"Finance and Insurance","53":"Real Estate and Rental and Leasing","54":"Professional, Scientific, and Technical Services",
           "55":"Management of Companies and Enterprises","56":"Administrative and Support and Waste Management and Remediation Services",
           "61":"Educational Services","62":"Health Care and Social Assistance","64":"Health Care and Social Assistance",
           "71":"Arts, Entertainment, and Recreation","72":"Accommodation and Food Services","73":"Accommodation and Food Services",
           "81":"Other Services (except Public Administration)","92":"Public Administration","99":"Public Administration" }

NAICS code is used for classification of business establishments by type of economic activity. The NAICS_CODE value in data points are replaced with actual values fetched from NAICS website. 

Because the EMPLOYER_STATE and WORKSITE_STATE had more of categorical variables, they were replaced with appropriate US regions.

In [17]:
h1b_df["EMPLOYER_STATE"].replace(state_map, inplace=True)
h1b_df["WORKSITE_STATE"].replace(state_map, inplace=True)
h1b_df['NAICS_CODE_UPTD'] = h1b_df['NAICS_CODE'].astype(str).str[0:2]
h1b_df["NAICS_CODE_UPTD"].replace(ind_map, inplace=True)

h1b_df=h1b_df.rename(columns = {'NAICS_CODE_UPTD':'EMPLOYER_INDUSTRY'})
h1b_df=h1b_df.rename(columns = {'EMPLOYER_STATE':'EMPLOYER_REGION'})
h1b_df=h1b_df.rename(columns = {'WORKSITE_STATE':'WORKSITE_REGION'})

#### Data Encoding on categorical features

In [18]:
h1b_df['CASE_STATUS'] = h1b_df['CASE_STATUS'].map({'CERTIFIED': 1, 'DENIED': 0})
X=h1b_df[h1b_df.columns.difference(['CASE_STATUS'])]
y=h1b_df['CASE_STATUS']


from sklearn import preprocessing
le = preprocessing.LabelEncoder()
for column_name in X.columns:
    if X[column_name].dtype == object:
        X[column_name] = le.fit_transform(X[column_name])
    else:
        pass

### PREDICTIVE MODELLING

#### Model Selection

In [19]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [20]:
warnings.filterwarnings("ignore", category=DeprecationWarning) 

from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

param_grid = {"n_estimators": [100, 200], "max_features": ['auto', 'sqrt'], "min_samples_split": [2, 5], "bootstrap": [True, False]}

model = RandomForestClassifier(random_state=0)
grid = GridSearchCV(estimator=model, param_grid=param_grid)
grid.fit(X_train, y_train)

print(grid.best_score_)
print(grid.best_params_)

0.9876009612175423
{'bootstrap': True, 'max_features': 'auto', 'min_samples_split': 5, 'n_estimators': 200}


#### Model Evaluation

In [21]:
train_z = grid.predict(X_train)
train_z_prob = grid.predict_proba(X_train)[:,1]

test_z = grid.predict(X_test)
test_z_prob = grid.predict_proba(X_test)[:,1]

In [23]:
from sklearn.metrics import accuracy_score, roc_auc_score

print("model accuracy on train set: {}".format(accuracy_score(y_train, train_z)))
print("model ROC AUC: {}".format(roc_auc_score(y_train, train_z_prob)))

print("model accuracy on test set: {}".format(accuracy_score(y_test, test_z)))
print("model ROC AUC: {}".format(roc_auc_score(y_test, test_z_prob)))

model accuracy on train set: 0.989636873372939
model ROC AUC: 0.9980256738902922
model accuracy on test set: 0.9845804685935519
model ROC AUC: 0.6833123353043402
